Team Jedward Datathon project!

In [2]:
#import values
import pandas as pd

In [3]:
#importing csv files to dataframes
customers_df = pd.read_csv("the_data\olist_customers_dataset.csv")
geolocation_df = pd.read_csv("the_data\olist_geolocation_dataset.csv")
order_items_df = pd.read_csv("the_data\olist_order_items_dataset.csv")
order_payment_df = pd.read_csv("the_data\olist_order_payments_dataset.csv")
order_reviews_df = pd.read_csv("the_data\olist_order_reviews_dataset.csv")
orders_df = pd.read_csv("the_data\olist_orders_dataset.csv")
products_df = pd.read_csv("the_data\olist_products_dataset.csv")
sellers_df = pd.read_csv("the_data\olist_sellers_dataset.csv")

In [4]:
order_reviews_df = order_reviews_df.fillna(0)

print(order_reviews_df.isnull().sum())

review_id                  0
order_id                   0
review_score               0
review_comment_title       0
review_comment_message     0
review_creation_date       0
review_answer_timestamp    0
dtype: int64


In [17]:
print(customers_df.duplicated().sum())

geolocation_df = geolocation_df.drop_duplicates(subset=['geolocation_zip_code_prefix'], keep='first')
geolocation_csv_data = geolocation_df.to_csv('cleaned_data/geolocation_dataset_cleaned.csv', index = False)

0
